In [1]:
import pandas as pd
import numpy as np

In [27]:
features_regional = np.load('features_regional.npy')
features_shape = np.load('features_shape.npy')
features_stats = np.load('features_stats.npy')
features_histogram = np.load('features_histogram.npy')
features_texture = np.load('texture_flat_1.npy')

features = np.concatenate((features_regional, features_shape,
                           features_stats,features_histogram,np.nan_to_num(features_texture)),axis=1)
features.shape

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 1472 and the array at index 4 has size 1

In [13]:
from sklearn import preprocessing

labels = pd.read_csv('../data/ATR_GT_Training.csv', header=None)

In [14]:
X = features[:1005]
y = labels.iloc[:1005, 1]
y_bin = (y!=0).astype(int)

In [15]:
X_scaled = preprocessing.scale(X)

In [16]:
classifiers=[]

In [17]:
from sklearn import svm
classifiers.append(['default svm', svm.SVC()])

In [18]:
from sklearn.linear_model import LogisticRegression
classifiers.append(['logistic reg', LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')])

In [19]:
from sklearn.ensemble import AdaBoostClassifier
classifiers.append(['adaboost', AdaBoostClassifier(n_estimators=100)])

In [20]:
from sklearn.neighbors import KNeighborsClassifier
classifiers.append(['knn 3', KNeighborsClassifier(n_neighbors=3)])

In [21]:
# Use a Multi-layer Perceptron classifier . It trains via backpropagation 
from sklearn.neural_network import MLPClassifier
classifiers.append(['NN', MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)])

In [22]:
from sklearn.model_selection import cross_val_score
def cross_validate(classifier,X,y,name,cv_=5):
    scores = cross_val_score(classifier, X, y, cv=cv_)
    print('Clf: {}\nAccuracy: {:0.2f} +/- {:0.2f}'.format(name, scores.mean(), scores.std() * 2))

In [26]:
import warnings
warnings.filterwarnings('ignore')
for classifier_name, classifier in classifiers:
    cross_validate(classifier,X_scaled,y_bin,classifier_name)

Clf: default svm
Accuracy: 0.79 +/- 0.06
Clf: logistic reg
Accuracy: 0.78 +/- 0.07
Clf: adaboost
Accuracy: 0.84 +/- 0.08
Clf: knn 3
Accuracy: 0.82 +/- 0.08
Clf: NN
Accuracy: 0.82 +/- 0.06


In [ ]:
from sklearn.ensemble import VotingClassifier
eclf = VotingClassifier(estimators=classifiers, voting='hard')
cross_validate(eclf,X_scaled,y_bin,'ensemble')

In [ ]:
eclf = eclf.fit(X, y)
pred = eclf.predict(features[1005:])
filenames = df_regional['file'].tolist()[1005:]
pred_df = {'file': filenames, 'prediction': pred}
pd.DataFrame(pred_df).to_csv('predictions.csv')